In [1]:
import pandas as pd
import numpy as np
from grouprecommender import GroupRecommender

Using TensorFlow backend.


# Loading the Dataset

In [2]:
poorly_formatted_rows = [2120259, 2446317, 11141080, 11152098, 11152401, 11882086, 12902538, 12935043, 17589538]
df = pd.read_csv('lastfm_data/userid-timestamp-artid-artname-traid-traname.tsv', header=None, skiprows=poorly_formatted_rows, sep='\t')
df.columns = ['user_id', 'timestamp', 'artist_id', 'artist_name', 'track_id', 'track_name']
df.dropna(inplace=True)

# Group Recommendations

In [3]:
gr = GroupRecommender('utility_matrix.pickle',
                      dataset=df,
                      pickled_model_path='model.pickle',
                      embedding_model_path='embedding_model.yaml',
                      model_weights_path='embedding_model.h5',
                      embedding_space_path='embedding.npy',
                      dicts_path='song_dicts.pickle')

## Obtaining Group Recommendations

In [4]:
user_ids = np.random.choice(df['user_id'].unique(), 3, replace=False)
max_recommendations = 10
playlist = gr.full_recommendation(user_ids, max_recommendations, df, method='item2vec')

In [5]:
print("Recommended Songs for the Group:", user_ids)
for i, track in enumerate(playlist):
    print(str(i) + ')' + track[0] + ' - ' + track[1])

Recommended Songs for the Group: ['user_000167' 'user_000658' 'user_000213']
0)Sex Pistols - God Save The Queen
1)Placebo - Song To Say Goodbye
2)Uriah Heep - Lady In Black
3)Judas Priest - A Touch Of Evil
4)Eddie Vedder - Society
5)The Who - Baba O'Riley
6)Marilyn Manson - Heart-Shaped Glasses (When The Heart Guides The Hand)
7)The Chemical Brothers - Do It Again
8)Sly & The Family Stone - If You Want Me To Stay
9)The 69 Eyes - Velvet Touch
10)Cat Power - Maybe Not


## Evaluating Recommendations

In [6]:
#TODO: implement evaluation method
track_ids = [track.track_id for track in playlist]
playlist_indexes = np.where(np.in1d(df['track_id'].dropna().unique(), track_ids))[0].tolist()
user_indexes = np.where(np.in1d(df['user_id'].dropna().unique(), user_ids))[0].tolist()
eval_score, similarity_score = gr.evaluate(user_indexes, playlist_indexes, method='custom')
print(eval_score, similarity_score)

11.5133219996 0.358433609422


In [7]:
playlist_indexes

[6739, 9712, 12408, 18477, 20849, 29185, 50993, 51379, 140789, 172434, 226320]

## Creating Group Playlist

In [8]:
import spotipy
import pprint
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '5de79a00f1b9475098f277ce2a609120'
client_secret = '909633d1415345fe89882b6e398390df'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id,
                                                          client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
spotify_track_ids = []
for track in playlist:
    search_str = track[0] + ' ' + track[1]
    result = sp.search(search_str, limit=1)
    spotify_track_ids.append(result['tracks']['items'][0]['id'])

In [10]:
playlist_name = 'Group recommendations for '
playlist_name += ', '.join(user_ids)

In [11]:
username = '11130460071'
scope='playlist-modify-public'
token = util.prompt_for_user_token(username, 
                                   client_id=client_id, 
                                   client_secret=client_secret, 
                                   redirect_uri='https://github.com/alexing10/musicmagal',
                                   scope=scope)

In [12]:
sp = spotipy.Spotify(auth=token)
playlist = sp.user_playlist_create(username, playlist_name)
sp.user_playlist_add_tracks(username, playlist['id'], spotify_track_ids)
print("boom!")

boom!


In [19]:
playlist_id = '0ChGv9bC33XK1zLtA78y9F'
sp.user_playlist_add_tracks(username, playlist_id, spotify_track_ids)

{'snapshot_id': 'oRoC3cCOO7mnauSmpMKZpVtSHgzRSSjGlAFVyyajNmNXYltq4fC0QA65082rDFtp'}